In [95]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [96]:
# business data at LA level, number and per 1000
businesses_la = pd.read_csv("../data/scotland/businesses_LA.csv")
businesses_la.rename(columns={"local authority: district / unitary (as of April 2021)": "Local authority", "Total": "Number of businesses"}, inplace=True)

pop_est = pd.read_csv("../data/scotland/mid-year-pop-est-2020.csv")
pop_est.rename(columns={"All Ages": "All ages"}, inplace=True)

businesses_la = pd.merge(businesses_la, pop_est, left_on="Local authority", right_on="Area name")
businesses_la = businesses_la.loc[:, ["Local authority", "Area code", "Number of businesses", "All ages"]].copy()
businesses_la["Number of businesses"] = businesses_la["Number of businesses"].astype(int)
businesses_la["All ages"] = businesses_la["All ages"].str.replace(",", "").astype(int)
businesses_la["Businesses per 1000 people"] = ((businesses_la["Number of businesses"] / businesses_la["All ages"]) * 1000).astype(int)

businesses_la

,Local authority,Area code,Number of businesses,All ages,Businesses per 1000 people
0,Aberdeen City,S12000033,11140,229060,48
1,Aberdeenshire,S12000034,15355,260780,58
2,Angus,S12000041,4640,115820,40
3,Argyll and Bute,S12000035,4820,85430,56
4,City of Edinburgh,S12000036,23875,527620,45
5,Clackmannanshire,S12000005,1360,51290,26
6,Dumfries and Galloway,S12000006,7550,148290,50
7,Dundee City,S12000042,4830,148820,32
8,East Ayrshire,S12000008,4080,121600,33
9,East Dunbartonshire,S12000045,3415,108750,31


In [97]:
# simd - generalise to LA on all domains
simd = pd.read_csv("../data/scotland/SIMD_2020.csv")
dz_lookup = pd.read_csv("../data/scotland/2011_dz_lookup.csv")
simd_decile = simd.loc[simd["Units"] == "Decile"].copy()

# creating a dataframe for each domain
domains = simd_decile['SIMD Domain'].unique()

def create_domain_df(units_df):
    dfs = []
    for domain in domains:
        df = units_df.loc[simd["SIMD Domain"] == domain].copy()
        df = df.merge(dz_lookup, left_on="FeatureCode", right_on="DZ2011_Code")
        df = df.groupby('LA_Name')['Value'].mean().reset_index()
        df["Value"] = df["Value"].round().astype(int)
        df.rename(columns={"Value": f"{domain} decile"}, inplace=True)
        dfs.append(df)

    return dfs

domain_dfs = create_domain_df(simd_decile) 

simd_simd = domain_dfs[0]
simd_access = domain_dfs[1]
simd_health = domain_dfs[2]   
simd_crime = domain_dfs[3]   
simd_edu = domain_dfs[4]   
simd_employment = domain_dfs[5]   
simd_housing = domain_dfs[6]   
simd_income = domain_dfs[7]   

named_domain_dfs = [simd_simd, simd_access, simd_health, simd_crime, simd_edu, simd_employment, simd_housing, simd_income]
merging_dfs = [df.set_index('LA_Name') for df in named_domain_dfs]
simd_la = pd.concat(merging_dfs, axis=1).reset_index()
simd_la

,LA_Name,SIMD decile,Access To Services decile,Health decile,Crime decile,Education Skills And Training decile,Employment decile,Housing decile,Income decile
0,Aberdeen City,7,7,7,5,6,6,5,7
1,Aberdeenshire,7,4,8,7,7,7,7,7
2,Angus,6,5,6,6,6,6,7,6
3,Argyll and Bute,6,4,6,7,6,6,5,6
4,City of Edinburgh,7,7,7,5,7,7,4,7
5,Clackmannanshire,5,5,5,5,5,5,6,5
6,Dumfries and Galloway,5,5,6,6,6,6,7,5
7,Dundee City,4,7,4,5,4,4,4,4
8,East Ayrshire,4,4,4,5,5,5,6,5
9,East Dunbartonshire,7,5,7,7,8,7,7,7


In [98]:
charities = pd.read_csv("../data/scotland/local_charities_final.csv")
postcodes = pd.read_csv("../data/scotland/postcode_lookup.csv", low_memory=False)
postcodes.drop_duplicates(subset="Postcode", inplace=True)

char_pc = pd.merge(charities, postcodes, left_on="Postcode", right_on="Postcode", how="left")
char_pc_la = pd.merge(char_pc, dz_lookup, left_on="DataZone2011Code", right_on="DZ2011_Code", how="left")

char_pc_la = char_pc_la.replace(to_replace="Perth & Kinross", value="Perth and Kinross")
char_pc_la = char_pc_la.replace(to_replace="Aberdeen", value="Aberdeen City")
char_pc_la = char_pc_la.replace(to_replace="Argyll & Bute", value="Argyll and Bute")
char_pc_la = char_pc_la.replace(to_replace="Dumfries & Galloway", value="Dumfries and Galloway")
char_pc_la = char_pc_la.replace(to_replace="Western Isles", value="Na h-Eileanan Siar")

char_pc_la = char_pc_la.loc[(char_pc_la['LA_Name'] == char_pc_la['Main Operating Location']) | (char_pc_la['LA_Name'].isna())].copy()
char_pc_la.drop(columns="LA_Name", inplace=True)
char_pc_la.rename(columns={"Main Operating Location": "LA_Name"}, inplace=True)

char_num_by_la = char_pc_la.groupby("LA_Name")['Charity Number'].count().reset_index()
char_num_by_la

# get per 1000 population
char_per_1000 = pd.merge(char_num_by_la, pop_est, left_on="LA_Name", right_on="Area name")
char_per_1000.rename(columns={"Charity Number": "Number of charities"}, inplace=True)
char_per_1000["All ages"] = char_per_1000["All ages"].str.replace(",", "").astype(int)
char_per_1000["Charities per 1000 people"] = ((char_per_1000["Number of charities"] / char_per_1000["All ages"]) * 1000).astype(int)

char_la = char_per_1000.loc[:, ["LA_Name", "Area code", "Number of charities", "All ages", "Charities per 1000 people"]].copy()

# merging what I have
la_info = businesses_la.merge(simd_la, left_on="Local authority", right_on="LA_Name")
la_info = la_info.merge(char_la, left_on="LA_Name", right_on="LA_Name")
la_info = la_info.loc[:, ["Local authority", "Number of businesses", "Businesses per 1000 people", "Number of charities", "Charities per 1000 people", "SIMD decile"]].copy()
#  "Access To Services decile", "Health decile", "Crime decile", "Education Skills And Training decile", "Employment decile", "Housing decile", "Income decile"

la_info
# la_info.to_csv("../data/scotland/la_info_part_done.csv", index=False)


,Local authority,Number of businesses,Businesses per 1000 people,Number of charities,Charities per 1000 people,SIMD decile
0,Aberdeen City,11140,48,354,1,7
1,Aberdeenshire,15355,58,1004,3,7
2,Angus,4640,40,390,3,6
3,Argyll and Bute,4820,56,636,7,6
4,City of Edinburgh,23875,45,1025,1,7
5,Clackmannanshire,1360,26,124,2,5
6,Dumfries and Galloway,7550,50,671,4,5
7,Dundee City,4830,32,247,1,4
8,East Ayrshire,4080,33,228,1,4
9,East Dunbartonshire,3415,31,160,1,7


In [99]:
# ccch number and per 1000 people for each LA 
ccch_aberdeen_city = pd.read_csv("../data/aberdeen_city/ccch_aberdeen_city.csv")
ccch_aberdeenshire = pd.read_csv("../data/aberdeenshire/ccch_aberdeenshire.csv")
ccch_angus = pd.read_csv("../data/angus/ccch_angus.csv")
ccch_argyll_and_bute = pd.read_csv("../data/argyll_and_bute/ccch_argyll_and_bute.csv")
ccch_city_of_edinburgh = pd.read_csv("../data/city_of_edinburgh/ccch_city_of_edinburgh.csv")
ccch_clackmannanshire = pd.read_csv("../data/clackmannanshire/ccch_clackmannanshire.csv")
ccch_dumfries_and_galloway = pd.read_csv("../data/dumfries_and_galloway/ccch_dumfries_and_galloway.csv")
ccch_dundee_city = pd.read_csv("../data/dundee_city/ccch_dundee_city.csv")
ccch_east_ayrshire = pd.read_csv("../data/east_ayrshire/ccch_east_ayrshire.csv")
ccch_east_dunbartonshire = pd.read_csv("../data/east_dunbartonshire/ccch_east_dunbartonshire.csv")
ccch_east_lothian = pd.read_csv("../data/east_lothian/ccch_east_lothian.csv")
ccch_east_renfrewshire = pd.read_csv("../data/east_renfrewshire/ccch_east_renfrewshire.csv")
ccch_falkirk = pd.read_csv("../data/falkirk/ccch_falkirk.csv")
ccch_fife = pd.read_csv("../data/fife/ccch_fife.csv")
ccch_glasgow_city = pd.read_csv("../data/glasgow_city/ccch_glasgow_city.csv")
ccch_highland = pd.read_csv("../data/highland/ccch_highland.csv")
ccch_inverclyde = pd.read_csv("../data/inverclyde/ccch_inverclyde.csv")
ccch_moray = pd.read_csv("../data/moray/ccch_moray.csv")
ccch_midlothian = pd.read_csv("../data/midlothian/ccch_midlothian.csv")
ccch_na_h_eileanan_siar = pd.read_csv("../data/na_h-eileanan_siar/ccch_na_h-eileanan_siar.csv")
ccch_north_ayrshire = pd.read_csv("../data/north_ayrshire/ccch_north_ayrshire.csv")
ccch_north_lanarkshire = pd.read_csv("../data/north_lanarkshire/ccch_north_lanarkshire.csv")
ccch_orkney_islands = pd.read_csv("../data/orkney_islands/ccch_orkney_islands.csv")
ccch_perth_and_kinross = pd.read_csv("../data/perth_and_kinross/ccch_perth_and_kinross.csv")
ccch_renfrewshire = pd.read_csv("../data/renfrewshire/ccch_renfrewshire.csv")
ccch_scottish_borders = pd.read_csv("../data/scottish_borders/ccch_scottish_borders.csv")
ccch_shetland_islands = pd.read_csv("../data/shetland_islands/ccch_shetland_islands.csv")
ccch_south_ayrshire = pd.read_csv("../data/south_ayrshire/ccch_south_ayrshire.csv")
ccch_south_lanarkshire = pd.read_csv("../data/south_lanarkshire/ccch_south_lanarkshire.csv")
ccch_stirling = pd.read_csv("../data/stirling/ccch_stirling.csv")
ccch_west_dunbartonshire = pd.read_csv("../data/west_dunbartonshire/ccch_west_dunbartonshire.csv")
ccch_west_lothian = pd.read_csv("../data/west_lothian/ccch_west_lothian.csv")


names_list = ["Aberdeen City", "Aberdeenshire", "Angus", "Argyll and Bute", "City of Edinburgh", "Clackmannanshire", "Dumfries and Galloway", "Dundee City", "East Ayrshire", "East Dunbartonshire", "East Lothian", "East Renfrewshire", "Falkirk", "Fife", "Glasgow City", "Highland", "Inverclyde", "Midlothian", "Moray", "Na h-Eileanan Siar", "North Ayrshire", "North Lanarkshire", "Orkney Islands", "Perth and Kinross", "Renfrewshire", "Scottish Borders", "Shetland Islands", "South Ayrshire", "South Lanarkshire", "Stirling", "West Dunbartonshire", "West Lothian"]
ccch_list = [ccch_aberdeen_city, ccch_aberdeenshire, ccch_angus, ccch_argyll_and_bute, ccch_city_of_edinburgh, ccch_clackmannanshire, ccch_dumfries_and_galloway, ccch_dundee_city, ccch_east_ayrshire, ccch_east_dunbartonshire, ccch_east_lothian, ccch_east_renfrewshire, ccch_falkirk, ccch_fife, ccch_glasgow_city, ccch_highland, ccch_inverclyde, ccch_midlothian, ccch_moray, ccch_na_h_eileanan_siar, ccch_north_ayrshire, ccch_north_lanarkshire, ccch_orkney_islands, ccch_perth_and_kinross, ccch_renfrewshire, ccch_scottish_borders, ccch_shetland_islands, ccch_south_ayrshire, ccch_south_lanarkshire, ccch_stirling, ccch_west_dunbartonshire, ccch_west_lothian]
fixed_ccch_list = []


def fix_ccch_df(ccch_df, i):
    ccch_df = ccch_df.loc[:, ["name", "postcode"]].copy()
    ccch_df.rename(columns={"name": "Name", "postcode": "Postcode"}, inplace=True)
    ccch_df["Local authority"] = names_list[i]
    fixed_ccch_list.append(ccch_df)
    return fixed_ccch_list


for x, ccch in enumerate(ccch_list):
    fix_ccch_df(ccch, x)
    

ccch_all = pd.concat(fixed_ccch_list)
ccch_all

ccch_num_by_la = ccch_all.groupby("Local authority")["Name"].count().reset_index()

ccch_per_1000 = pd.merge(ccch_num_by_la, pop_est, left_on="Local authority", right_on="Area name")
ccch_per_1000.rename(columns={"Name": "Number of community spaces"}, inplace=True)
ccch_per_1000["All ages"] = ccch_per_1000["All ages"].str.replace(",", "").astype(int)
ccch_per_1000["Community spaces per 1000 people"] = ((ccch_per_1000["Number of community spaces"] / char_per_1000["All ages"]) * 1000).astype(float).round(1)

ccch_per_1000 = ccch_per_1000.loc[:, ["Local authority", "Number of community spaces", "Community spaces per 1000 people"]].copy()
ccch_per_1000


,Local authority,Number of community spaces,Community spaces per 1000 people
0,Aberdeen City,86,0.4
1,Aberdeenshire,162,0.6
2,Angus,63,0.5
3,Argyll and Bute,105,1.2
4,City of Edinburgh,158,0.3
5,Clackmannanshire,46,0.9
6,Dumfries and Galloway,128,0.9
7,Dundee City,70,0.5
8,East Ayrshire,63,0.5
9,East Dunbartonshire,43,0.4


In [100]:
# maos per 100k people for each LA
maos = pd.read_csv("../data/scotland/maos.csv")
maos = maos.merge(postcodes, left_on="postcode", right_on="Postcode")
maos_la = pd.merge(maos, dz_lookup, left_on="DataZone2011Code", right_on="DZ2011_Code", how="left")
maos_la = maos_la.loc[:, ["name", "postcode", "LA_Name"]].copy()

maos_la = maos_la.rename(columns={"LA_Name": "Local authority"})

maos_by_la = maos_la.groupby("Local authority")['name'].count().reset_index()

maos_per_100k = pd.merge(maos_by_la, pop_est, left_on="Local authority", right_on="Area name", how="right")
maos_per_100k.rename(columns={"name": "Number of MAOs"}, inplace=True)
maos_per_100k["All ages"] = maos_per_100k["All ages"].str.replace(",", "").astype(int)
maos_per_100k["MAOs per 100k people"] = ((maos_per_100k["Number of MAOs"] / maos_per_100k["All ages"]) * 100000).astype(float).round(1)

maos_per_100k = maos_per_100k.loc[:, ["Area name", "Number of MAOs", "MAOs per 100k people"]].copy()
maos_per_100k.rename(columns={"Area name": "Local authority"}, inplace=True)
maos_per_100k = maos_per_100k.fillna(0.0)
maos_per_100k

,Local authority,Number of MAOs,MAOs per 100k people
0,Aberdeen City,1.0,0.4
1,Aberdeenshire,18.0,6.9
2,Angus,5.0,4.3
3,Argyll and Bute,4.0,4.7
4,City of Edinburgh,33.0,6.3
5,Clackmannanshire,1.0,1.9
6,Dumfries and Galloway,21.0,14.2
7,Dundee City,2.0,1.3
8,East Ayrshire,6.0,4.9
9,East Dunbartonshire,3.0,2.8


In [101]:
# shs percentage for the LA
shs = pd.read_csv("../data/scotland/SHS Data Explorer.csv")
shs = shs.loc[:, ["Council", "All"]].copy()
shs.rename(columns={"All": "I could rely on someone in my neighbourhood to help me (%)"}, inplace=True)
shs.drop(shs[shs["Council"] == "Scotland"].index, inplace=True)

shs["Council"].at[11] = "City of Edinburgh"
shs.at[19, "Council"] = "Na h-Eileanan Siar"
shs = shs.sort_values("Council").reset_index(drop=True)
shs

,Council,I could rely on someone in my neighbourhood to help me (%)
0,Aberdeen City,71.5
1,Aberdeenshire,88.3
2,Angus,79.0
3,Argyll and Bute,92.2
4,City of Edinburgh,82.0
5,Clackmannanshire,85.1
6,Dumfries and Galloway,90.7
7,Dundee City,83.1
8,East Ayrshire,75.3
9,East Dunbartonshire,88.3


In [102]:
dt = pd.read_csv("../data/scotland/dt_complete.csv")
dt = dt.merge(postcodes, left_on="postcode", right_on="Postcode")
dt_la = pd.merge(dt, dz_lookup, left_on="DataZone2011Code", right_on="DZ2011_Code", how="left")
dt_la = dt_la.loc[:, ["name", "postcode", "LA_Name"]].copy()

dt_la = dt_la.rename(columns={"LA_Name": "Local authority"})

dt_by_la = dt_la.groupby("Local authority")['name'].count().reset_index()

dt_per_100k = pd.merge(dt_by_la, pop_est, left_on="Local authority", right_on="Area name", how="right")
dt_per_100k.rename(columns={"name": "Number of development trusts"}, inplace=True)
dt_per_100k["All ages"] = dt_per_100k["All ages"].str.replace(",", "").astype(int)
dt_per_100k["Development trusts per 100k people"] = ((dt_per_100k["Number of development trusts"] / dt_per_100k["All ages"]) * 100000).astype(float).round(1)

dt_per_100k = dt_per_100k.loc[:, ["Area name", "Number of development trusts", "Development trusts per 100k people"]].copy()
dt_per_100k.rename(columns={"Area name": "Local authority"}, inplace=True)
dt_per_100k = dt_per_100k.fillna(0.0)
dt_per_100k

,Local authority,Number of development trusts,Development trusts per 100k people
0,Aberdeen City,2.0,0.9
1,Aberdeenshire,13.0,5.0
2,Angus,1.0,0.9
3,Argyll and Bute,33.0,38.6
4,City of Edinburgh,10.0,1.9
5,Clackmannanshire,4.0,7.8
6,Dumfries and Galloway,20.0,13.5
7,Dundee City,0.0,0.0
8,East Ayrshire,8.0,6.6
9,East Dunbartonshire,4.0,3.7


In [103]:
# life expectancy, male and female avg for each LA
le = pd.read_csv("../data/scotland/scotpho_data_extract.csv")
female_le = le.loc[(le['indicator'] == "Life expectancy, females") & (le['area_type'] == "Council area") & (le['period'] == '2017 to 2019 (3 year aggregate)')].copy()
female_le = female_le.loc[:, ["area_name", "measure"]].copy()
female_le.rename(columns={"measure": "Female life expectancy"}, inplace=True)
female_le['Female life excpectancy'] = female_le['Female life expectancy'].apply(lambda x: round(x))
   
male_le = le.loc[(le['indicator'] == "Life expectancy, males") & (le['area_type'] == "Council area") & (le['period'] == '2017 to 2019 (3 year aggregate)')].copy()
male_le = male_le.loc[:, ["area_name", "measure"]].copy()
male_le.rename(columns={"measure": "Male life expectancy"}, inplace=True)
male_le['Male life expectancy'] = male_le['Male life expectancy'].apply(lambda x: round(x))

le = female_le.merge(male_le, left_on="area_name", right_on="area_name")
le.at[1, "area_name"] = "Dumfries and Galloway"
le.at[20, "area_name"] = "Argyll and Bute"
le.at[29, "area_name"] = "Perth and Kinross"
# le.at[1, "area_name"] = "Dumfries and Galloway"
le


,area_name,Female life expectancy,Female life excpectancy,Male life expectancy
0,Clackmannanshire,80.70,81,77
1,Dumfries and Galloway,81.44,81,78
2,East Ayrshire,79.81,80,76
3,East Lothian,82.72,83,79
4,East Renfrewshire,83.96,84,80
5,Na h-Eileanan Siar,83.39,83,78
6,Falkirk,80.46,80,77
7,Highland,81.91,82,78
8,Inverclyde,79.01,79,75
9,Midlothian,81.50,82,78


In [104]:
la_info = pd.merge(la_info, ccch_per_1000, left_on="Local authority", right_on="Local authority")
la_info = la_info.merge(shs, left_on="Local authority", right_on="Council")
la_info.drop(columns="Council", inplace=True)
la_info = pd.merge(la_info, maos_per_100k, left_on="Local authority", right_on="Local authority", how="left")
la_info = pd.merge(la_info, le, left_on="Local authority", right_on="area_name")
la_info = pd.merge(la_info, dt_per_100k, left_on="Local authority", right_on="Local authority")

In [105]:
la_info_complete = la_info.copy()
la_info_complete = la_info_complete.fillna(0)

la_info_complete = la_info_complete.loc[:, ["Local authority", "SIMD decile", "Female life expectancy", "Male life expectancy", "Number of businesses", "Number of charities", "Number of community spaces", "Number of MAOs", "Number of development trusts", "Businesses per 1000 people", "Charities per 1000 people", "Community spaces per 1000 people", "MAOs per 100k people", "I could rely on someone in my neighbourhood to help me (%)", "Devel"]].copy()

In [106]:
# la_info_complete.to_csv("../data/scotland/la_info.csv", index=False)

In [107]:
la_info_complete

,Local authority,SIMD decile,Female life expectancy,Male life expectancy,Number of businesses,Number of charities,Number of community spaces,Number of MAOs,Businesses per 1000 people,Charities per 1000 people,Community spaces per 1000 people,MAOs per 100k people,I could rely on someone in my neighbourhood to help me (%)
0,Aberdeen City,7,81.40,77,11140,354,86,1.0,48,1,0.4,0.4,71.5
1,Aberdeenshire,7,82.52,79,15355,1004,162,18.0,58,3,0.6,6.9,88.3
2,Angus,6,82.26,78,4640,390,63,5.0,40,3,0.5,4.3,79.0
3,Argyll and Bute,6,81.69,78,4820,636,105,4.0,56,7,1.2,4.7,92.2
4,City of Edinburgh,7,82.50,78,23875,1025,158,33.0,45,1,0.3,6.3,82.0
5,Clackmannanshire,5,80.70,77,1360,124,46,1.0,26,2,0.9,1.9,85.1
6,Dumfries and Galloway,5,81.44,78,7550,671,128,21.0,50,4,0.9,14.2,90.7
7,Dundee City,4,79.48,74,4830,247,70,2.0,32,1,0.5,1.3,83.1
8,East Ayrshire,4,79.81,76,4080,228,63,6.0,33,1,0.5,4.9,75.3
9,East Dunbartonshire,7,83.65,81,3415,160,43,3.0,31,1,0.4,2.8,88.3
